In [ ]:
import os
import glob
import cv2
import time
import copy
import pickle  # Log dictionary data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn.metrics as metrics

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # stateless functions
import torchvision.transforms as T
import torchvision.models as models

import multiprocessing
# We must import this explicitly, it is not imported by the top-level
# multiprocessing module.
import multiprocessing.pool

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score,confusion_matrix
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from datetime import datetime
from multiprocessing import Manager
from PIL import Image

import glob
import cv2
import time
import copy
import pickle  # Log dictionary data
import skimage.io
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn.metrics as metrics

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # stateless functions
import torchvision.transforms as T
import torchvision.models as models

from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import cohen_kappa_score,confusion_matrix
from PIL import Image

In [ ]:
"""Configuration in Common
"""
class CFG:
    debug = False
    device = torch.device('cuda')
    dtype = torch.float32
    nfolds = 4
    seed = 524
    TRAIN = '../yi_data/panda-16x128x128-tiles-data/train/'
    LABELS = '../data/train.csv'

"""Configuration for Stage One
"""
class CFG1:
    batch_size = 16    
    epochs = 16
    lr = 1e-4
    model_name = 'resnet_bc'
    num_classes = 1
    nworkers = 1    
    threshold = .32
    
"""Configuration for Stage Two
"""
class CFG2:
    batch_size = 16
    epochs = 16
    lr = 1e-4
    model_name = 'resnet'
    num_classes = 3
    nworkers = 1

# Datasets and Dataloader

In [ ]:
train = pd.read_csv(CFG.LABELS).set_index('image_id')
files = sorted(set([p[:32] for p in os.listdir(CFG.TRAIN)]))
train = train.loc[files].reset_index()

if CFG.debug:
    df = train.sample(n=50, random_state=CFG.seed).copy().reset_index(drop=True)
else:
    df = train.copy()

# Generate train/validation sets containing the same distribution of isup_grade
splits = StratifiedKFold(n_splits=CFG.nfolds, random_state=CFG.seed, shuffle=True)
splits = list(splits.split(df,df.isup_grade))
# Assign split index to training samples
folds_splits = np.zeros(len(df)).astype(np.int)
for i in range(CFG.nfolds):
    folds_splits[splits[i][1]] = i
df['split'] = folds_splits
df.head()

In [ ]:
# https://www.kaggle.com/yasufuminakama/panda-se-resnext50-regression-baseline
class TrainDataset(Dataset):
    """Prostate Cancer Biopsy Dataset"""
    
    def __init__(self, df, labels, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file
            root_dir (string): Path to the directory with all images
            transform (callable, optional): Optional transform to be applied on an image sample
        """
        # Shuffle dataframes with fixed seed; otherwise, validation set only get cancerous samples
        self.df = df
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # https://stackoverflow.com/questions/33369832/read-multiple-images-on-a-folder-in-opencv-python
        img_fns = [fn for fn in glob.glob(f"{CFG.TRAIN}/{self.df['image_id'][idx]}_*.png")]
        # As we use cv2, the color channel is BGR. https://stackoverflow.com/questions/50963283/python-opencv-imshow-doesnt-need-convert-from-bgr-to-rgb
        imgs = [cv2.imread(fn) for fn in img_fns]
        # (D,W,H)
        img = cv2.hconcat([cv2.vconcat([imgs[0], imgs[1], imgs[2], imgs[3]]),
                           cv2.vconcat([imgs[4], imgs[5], imgs[6], imgs[7]]),
                           cv2.vconcat([imgs[8], imgs[9], imgs[10], imgs[11]]),
                           cv2.vconcat([imgs[12], imgs[13], imgs[14], imgs[15]])])
        img = Image.fromarray(img)
        
        if self.transform:
            img = self.transform(img)
            
        label = self.labels[idx]
        return img, label

In [ ]:
def get_transforms(phase):
    assert phase in {'train', 'val'}
    
    if phase == 'train':
        return T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomRotation(15, fill=255),
            T.ToTensor(),
            T.Normalize(
                mean=[0.8776, 0.8186, 0.9090],
                std=[0.1659, 0.2507, 0.1357],
            ),
        ])
    else:
        return T.Compose([
            T.ToTensor(),
            T.Normalize(
                mean=[0.8776, 0.8186, 0.9090],
                std=[0.1659, 0.2507, 0.1357],
            ),
        ])

In [ ]:
"""
transform = get_transforms(phase='train')
train_dataset = TrainDataset(df.reset_index(drop=True),
                             df.reset_index(drop=True)['isup_grade'],
                             transform = get_transforms(phase='train'))
img, label = train_dataset[0]
#tiles = map(transform, tiles)
#print(list(tiles))
#print(label)
#print(tiles.shape)
#print(type(tiles[0]))
plt.imshow(img.permute(1,2,0))
#plt.show()
#plt.imshow(tiles[0].permute(1,2,0))
"""

In [ ]:
# Use fold idx as validation set
def data_loader_stage1(fold_idx):
    train_idx = df[df['split'] != fold_idx].index
    val_idx = df[df['split'] == fold_idx].index

    train_dataset = TrainDataset(df.loc[train_idx].reset_index(drop=True),
                                 df.loc[train_idx].reset_index(drop=True)['isup_grade'],
                                 transform = get_transforms(phase='train'))
    val_dataset = TrainDataset(df.loc[val_idx].reset_index(drop=True),
                               df.loc[val_idx].reset_index(drop=True)['isup_grade'],
                               transform = get_transforms(phase='val'))
    
    train_loader = DataLoader(train_dataset, batch_size=CFG1.batch_size, shuffle=True, num_workers=CFG1.nworkers)
    val_loader = DataLoader(val_dataset, batch_size=CFG1.batch_size, shuffle=False, num_workers=CFG1.nworkers)
    return train_loader, val_loader

def data_loader_stage2(fold_idx):
    train_idx = df[(df['split'] != fold_idx) & (df['isup_grade'] > 2)].index
    val_idx = df[(df['split'] == fold_idx) & (df['isup_grade'] > 2)].index

    train_dataset = TrainDataset(df.loc[train_idx].reset_index(drop=True),
                                 df.loc[train_idx].reset_index(drop=True)['isup_grade']-3,
                                 transform = get_transforms(phase='train'))
    val_dataset = TrainDataset(df.loc[val_idx].reset_index(drop=True),
                               df.loc[val_idx].reset_index(drop=True)['isup_grade']-3,
                               transform = get_transforms(phase='val'))
    
    train_loader = DataLoader(train_dataset, batch_size=CFG2.batch_size, shuffle=True, num_workers=CFG2.nworkers)
    val_loader = DataLoader(val_dataset, batch_size=CFG2.batch_size, shuffle=False, num_workers=CFG2.nworkers)
    return train_loader, val_loader

# Training

In [ ]:
def train_model(stage, model, fold, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    # Send the model to GPU/CPU
    model = model.to(device=CFG.device)
    
    train_acc_history = []
    val_acc_history = []
    loss_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            preds, targets = [], []

            if phase == 'train':
                model.train()   # Set model to training phase
            else:
                model.eval()    # Set model to evaluate phase
            
            avg_loss = 0.0
            running_corrects = 0
            
            print(' ', end='', flush=True)  # To workaround tqdm issue in multiprocess
            for inputs, labels in tqdm(dataloaders[phase],
                                       desc='[{}-{}] {}/{}({:5s})'.format(stage, fold, epoch+1,num_epochs,phase)):
                inputs = inputs.to(device=CFG.device, dtype=CFG.dtype)
                if stage == 1:
                    labels = torch.where(labels<3, torch.tensor(0), torch.tensor(1)).to(device=CFG.device, dtype=CFG.dtype)
                elif stage == 2:
                    labels = labels.to(device=CFG.device, dtype=torch.long)
                else:
                    assert False
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward, track history if only in training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs).squeeze()
                    #print(outputs.shape, labels.shape)
                    loss = criterion(outputs, labels)
                    #print(outputs)
                    if stage == 1:
                        pred = torch.where(outputs<CFG1.threshold,
                                           torch.tensor(0).to(device=CFG.device),
                                           torch.tensor(1).to(device=CFG.device)).to(device=CFG.device, dtype=torch.long)
                        labels = labels.to(dtype=torch.long)
                    #print(pred)
                    #print(labels)
                    elif stage == 2:
                        pred = torch.argmax(outputs, 1)
                        print(type(pred))
                        #print(pred.shape)
                    else:
                        assert False

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # Statistics
                avg_loss += loss.item()*(inputs.size(0)/len(dataloaders[phase].dataset))  # len(dataloaders[phase].dataset) not len(dataloaders[phase])
                running_corrects += torch.sum(pred == labels)
                #print('correts: ', torch.sum(pred == labels))
                preds.append(pred)
                targets.append(labels)
            
            # End of epoch
            with torch.no_grad():
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                if phase == 'val':
                    val_acc_history.append(epoch_acc)
                    # deep copy the model
                    if epoch_acc > best_acc:
                        best_acc = epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())
                    # Apply lr_scheduler
                    if scheduler is not None:
                        scheduler.step(avg_loss)
                else:
                    train_acc_history.append(epoch_acc)
                    loss_history.append(avg_loss)
                print('[{}-{}] {} Loss: {:4f} Acc: {:4f}'.format(stage, fold, phase, avg_loss, epoch_acc))
    
    time_elapsed = time.time() - since
    print('[{}-{}] Training complete in {:.0f}m {:0f}s'.format(stage, fold, time_elapsed//60, time_elapsed%60))
    print('[{}-{}] Best val Acc: {:4f}'.format(stage, fold, best_acc))
    print()
    
    model.load_state_dict(best_model_wts)
    return model, loss_history, train_acc_history, val_acc_history

## Two-Stage Classifier

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            

def initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False):
    """
    Params:
        feature_extract
            True - fine tunning
            False - fix the model
    """
    model_ft = None
    
    if model_name == 'alexnet':
        """AlexNet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
    elif model_name == 'resnet':
        """Resnet
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == 'resnet_bc':
        """Resnet
        """
        res_model = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(res_model, feature_extract)
        num_ftrs = res_model.fc.in_features
        res_model.fc = nn.Linear(num_ftrs, num_classes)
        model_ft = nn.Sequential(
                    res_model,
                    nn.Sigmoid(),  # Squeeze output within [0,1]
        )
    return model_ft


def train_stage1_layer(fold):
    model_ft = initialize_model(CFG1.model_name, CFG1.num_classes, use_pretrained=False)

    optimizer = optim.Adam(model_ft.parameters(),
                           lr=CFG1.lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, eps=1e-06)
    loader_train, loader_val = data_loader_stage1(fold)
    return train_model(1, model_ft, fold,
                       {'train': loader_train, 'val': loader_val},
                       nn.BCELoss(), optimizer, scheduler, CFG1.epochs)

def train_stage2_layer(fold):
    model_ft = initialize_model(CFG2.model_name, CFG2.num_classes, use_pretrained=False)
    optimizer = optim.Adam(model_ft.parameters(),
                           lr=CFG2.lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, eps=1e-06)
    loader_train, loader_val = data_loader_stage2(fold)
    return train_model(2, model_ft, fold,
                      {'train': loader_train, 'val': loader_val},
                      F.cross_entropy, optimizer, scheduler, CFG2.epochs)

## Multiprocessing

In [ ]:
class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

def progressor(fold):
    #print(f'stage{stage} fold{fold}')
    best_model_1, loss_history_1, train_acc_history_1, val_acc_history_1 = train_stage1_layer(fold)
    best_model_2, loss_history_2, train_acc_history_2, val_acc_history_2 = train_stage2_layer(fold)
    return {f'stage1_model_{fold}': best_model_1.to('cpu'),  # Don't save model as cuda
            f'stage1_loss_history_{fold}': loss_history_1,
            f'stage1_train_acc_history_{fold}': train_acc_history_1,
            f'stage1_val_acc_history_{fold}': val_acc_history_1,
            f'stage2_model_{fold}': best_model_2.to('cpu'),  # Don't save model as cuda
            f'stage2_loss_history_{fold}': loss_history_2,
            f'stage2_train_acc_history_{fold}': train_acc_history_2,
            f'stage2_val_acc_history_{fold}': val_acc_history_2}

## Start Training

In [ ]:
log_dict = {'batch_size_1': CFG1.batch_size,
            'batch_size_2': CFG2.batch_size,
            'epochs_1': CFG1.epochs,
            'epochs_2': CFG2.epochs,
            'learning_rate_1': CFG1.lr,
            'learning_rate_2': CFG2.lr,
            'model_1': CFG1.model_name,
            'model_2': CFG2.model_name,
            'nworkers_1': CFG1.nworkers,
            'nworkers_2': CFG2.nworkers,
            'nfolds': CFG.nfolds,
            'random_seed': CFG.seed}

result_list = list(MyPool(CFG.nfolds).map(progressor, range(CFG.nfolds)))

# Accumulate result from each process
for result in result_list:
    log_dict.update(result)

# Metrics
## Analyze Stage One

In [ ]:
targets, scores = [], []
for fold in range(CFG.nfolds):
    model_fd = log_dict[f'stage1_model_{fold}'].to(device=CFG.device, dtype=CFG.dtype)
    _, loader_val = data_loader_stage1(fold)
    for inputs, labels in tqdm(loader_val):
        inputs = inputs.to(device=CFG.device, dtype=CFG.dtype)
        labels = torch.where(labels<3, torch.tensor(0), torch.tensor(1)).to(device=CFG.device, dtype=CFG.dtype)
                
        # Forward, track history if only in training
        with torch.no_grad():
            outputs = model_fd(inputs).squeeze()
        targets.append(labels)
        scores.append(outputs)

t1 = torch.cat(targets).cpu()
s1 = torch.cat(scores).cpu()

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(t1, s1.view(-1))
roc_auc = metrics.auc(fpr, tpr)
log_dict['fpr'] = fpr
log_dict['tpr'] = tpr
log_dict['threshold'] = threshold


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

tn, fp, fn, tp = conf_mat.ravel()
print(f'sensitivity: {tp/(tp+fn)}')
print(f'specificity: {tn/(tn+fp)}')

## Analyze Stage Two

In [ ]:
masks, preds, targets = [], [], []
for fold in range(CFG.nfolds):
    model_fd1 = log_dict[f'stage1_model_{fold}'].to(device=CFG.device, dtype=CFG.dtype)
    model_fd2 = log_dict[f'stage2_model_{fold}'].to(device=CFG.device, dtype=CFG.dtype)
    _, loader_val = data_loader_stage2(fold)
    for inputs, labels in tqdm(loader_val):
        inputs = inputs.to(device=CFG.device, dtype=CFG.dtype)
                
        # Forward, track history if only in training
        with torch.no_grad():
            probs = model_fd1(inputs).squeeze()
            outputs = model_fd2(inputs)
            mask = torch.where(probs<CFG1.threshold,
                               torch.tensor(0).to(device=CFG.device),
                               torch.tensor(1).to(device=CFG.device))
            pred = torch.argmax(outputs, 1)
        masks.append(mask)
        targets.append(labels)
        preds.append(pred)

In [ ]:
p2 = torch.cat(preds).cpu()
t2 = torch.cat(targets).cpu()
m2 = torch.cat(masks).cpu()

kappa = cohen_kappa_score(t2[m2==1], p2[m2==1], weights='quadratic')
print(f'Kappa: {kappa}')
conf_mat = confusion_matrix(t2[m2==1], p2[m2==1])
#plt.matshow()
plt.figure(figsize=(14,7))
isup_labels = list(range(3,6))
sn.heatmap(conf_mat, annot=True, xticklabels=isup_labels, yticklabels=isup_labels)

## Log results

In [ ]:
log_file = f'{CFG1.model_name}-{CFG2.model_name}_{datetime.now().strftime("%m_%d_%Y_%H_%M")}.pkl'
with open(log_file, 'wb') as pkl_file:
    pickle.dump(log_dict, pkl_file)